Our own image compression schemes!

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import IPython.display

from cued_sf2_lab.familiarisation import load_mat_img, plot_image
from compression_schemes.dwt_funcs import *
from cued_sf2_lab.familiarisation import load_mat_img, plot_image
from compression_schemes.subjective_quality import ssim
from cued_sf2_lab.jpeg_dwt import *

In [ ]:
X, _ = load_mat_img(img='flamingo.mat', img_info='X')
X = X-128.0
opt_step, scaled, _, _, _ = diff_step_sizes(X, 256, n=5, target_rms = 10, rise_ratio=1.0)
print(f"optimal step size {opt_step}")
vlc, dhufftab, totalbits = jpegenc_dwt(X, 5,dcbits=9, steps = scaled, rise_ratio=1.0, opthuff=True)
print(totalbits)

In [ ]:
Z = jpegdec_dwt(vlc, 5, steps=scaled, rise_ratio = 1.0, hufftab = dhufftab, dcbits = 9)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,4))
plot_image(X, ax=ax1)
ax1.set(title="X")
plot_image(Z, ax=ax2)
ax2.set(title="Z")
print(ssim(X, Z))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# List of image filenames
image_list = ['flamingo.mat', 'lighthouse.mat', 'bridge.mat']  
rms_targets = np.linspace(6, 14, 10)

# Start plotting
plt.figure()

for img_name in image_list:
    # Load and preprocess
    X, _ = load_mat_img(img=img_name, img_info='X')
    X = X - 128.0

    bits = []
    rms_errors = []
    #estimates = []

    for target_rms in rms_targets:
        opt_step, scaled, _, _, _ = diff_step_sizes(X, 256, n=5, target_rms=target_rms, rise_ratio=1.0)
        #encoding
        vlc, dhufftab, totalbits = jpegenc_dwt(X, 5, dcbits=9, steps=scaled, rise_ratio=1.0, log=False, opthuff=True)
        bits.append(totalbits)
        #comparing to estimate
        #Y = nlevdwt(X, n=5)
        #Yq, dwtent= quantdwt(Y, scaled, rise_ratio=1.0)
        #Yq = np.round(Yq).astype(int)
        #_, estimated_bits = compression_ratio_for_DWT(X, Yq, dwtent)
        #estimates.append(int(round(estimated_bits)))
        #decoding
        Z = jpegdec_dwt(vlc, 5, steps=scaled, rise_ratio = 1.0, hufftab = dhufftab, dcbits = 9, log=False)
        actual_rms = np.std(X-Z)
        rms_errors.append(actual_rms)
        print(f"actual rms:{actual_rms},target rms: {target_rms}")
        #print(f"actual no. bits {totalbits}, estimated no.bits {estimated_bits:.0f}")
    # Plot curve for this image
    plt.plot(rms_errors, bits, 'o-', label=img_name.replace('.mat', ''))
    #plt.plot(rms_errors, estimates, '--', label = f"{img_name.replace('.mat', '')} (estimated bits)")
# Horizontal line at 5 kB
plt.axhline(y=40960, color='red', linestyle='--', label='5 kB = 40960 bits')

# Finalize plot
plt.xlabel('RMS error')
plt.ylabel('Total bits')
plt.title('Rate-Distortion Curve (DWT Compression)')
plt.grid(True)
plt.legend()
plt.show()


Testing an LBT scheme!